Получение и прверка матрицы гиперкомплексного числа

In [238]:
import sympy as sp
import numpy as np
import pandas as pd
from itertools import groupby
ar = np.array
np.set_printoptions(linewidth=150)
from functools import reduce

In [239]:
def get_idxs(n):
    ''' Матрица индексов для расстановки компонент гиперкомплексного числа.
        размер получаемой квадратной матрицы 2**n'''
    if n==0: return ar([0])
    a = get_idxs(n-1)
    b = a+2**(n-1)
    return chess(a,b)

def chess(a,b):
    ''' Получение  матрицы удвоенного размера при размещении малых матриц a и b  в шахматном порядке'''
    ab = np.hstack([a,b])
    ba = np.hstack([b,a])
    return np.vstack([ab,ba])

def rnd_f(n):
    ''' Выдает n-менрный случайный вектор с нормой 1'''
    r = np.random.rand(n)
    r /= (r*r).sum()**0.5
    return r

def eq_from_sum(s):
    ''' Выделяет из одного выражения уравнения для зависимостей знаков'''
    return [(a+b).factor().args[2] for a,b in ar(s.args).reshape(n//2,2)]

def get_dependens(req):
    ''' Собирает все зависимости для знаков'''
    dep = set()
    for eq in req:
        dep |= set(eq_from_sum(eq))
    return list(dep)

def to_bool_eq(dep):
    ''' Преобразует зависимости вида s0*s1+s2*s3 в логические уравнения'''
    eq = lambda a,b,c,d: a^b^c^d
    return [ eq(*dp.atoms()) for dp in dep]

def to_sign_mx(s):
    '''Преобразует решение логического уравнения в матрицу'''
    size = int(len(s)**0.5)
    signs = [s[sp.symbols(f's{i}')] for i in range(len(s))]
    return (ar(signs)*2-1).reshape(size,size)

def test_rot_mx(ixs,signs):
    ''' Тестирует матрицу поворота на случайном гиперкомплексном числе с нормой 1'''
    n = len(ixs)
    a = rnd_f(n)
    bs = a[ixs]*signs
    return abs(bs@bs.T-np.eye(len(bs))).sum()

In [240]:
k = 3 
#Размер матрицы
n = 2**k
a = ar(sp.var(f'a0:{n}'))
s = ar(sp.var(f's0:{n*n}')).reshape(n,n)
ixs = get_idxs(k)

In [241]:
bs = a[ixs]*s
eqs = bs@bs.T
req = eqs[np.eye(n)==0]
dep = get_dependens(req)

In [242]:
beq = to_bool_eq(dep)

In [243]:
ss = sp.satisfiable(reduce(sp.And,beq))
if ss is False: 
    raise Exception("Решение не найдено")

In [244]:
signs = to_sign_mx(ss)

In [245]:
test_rot_mx(ixs, signs)

0.0

In [246]:
# Компоненты марицы
signs*(ixs+1)

array([[-1, -2,  3,  4, -5,  6,  7, -8],
       [-2,  1, -4,  3,  6,  5, -8, -7],
       [-3, -4, -1, -2,  7,  8,  5,  6],
       [ 4, -3, -2,  1,  8, -7,  6, -5],
       [ 5,  6, -7,  8, -1,  2,  3,  4],
       [-6,  5, -8, -7, -2, -1,  4, -3],
       [-7, -8, -5,  6, -3, -4, -1,  2],
       [ 8, -7, -6, -5, -4,  3, -2, -1]])

Для размера матрицы больше 8 решение не находится. Скорее всего его нет.